In [4]:
# ==============================================================================
# SCRIPT DE ANÁLISE AVANÇADA - v3.0 (Caçador de Arbitragem Triangular)
# ==============================================================================

import pandas as pd
from sqlalchemy import create_engine
import json
from web3 import Web3
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings('ignore')

# --- PASSO 1: Carregar e Preparar os Dados (como antes) ---
print("--- Passo 1: Carregando e decodificando dados da Mainnet ---")
# Substitua pelo seu IP público
SERVER_IP = "34.79.99.180" 
db_connection_str = f'postgresql://admin:supersecret@{SERVER_IP}:5433/mempool_data'
df_final = pd.DataFrame()
try:
    db_engine = create_engine(db_connection_str)
    df_raw = pd.read_sql('SELECT "inputData", "to_address", "hash" FROM transactions', con=db_engine)
    
    # Decodificação (simplificada para focar na lógica do triângulo)
    router_abi_str = '[{"name":"swapExactTokensForTokens","inputs":[{"type":"uint256","name":"amountIn"},{"type":"address[]","name":"path"}],"type":"function"}]'
    router_abi = json.loads(router_abi_str)
    generic_contract = Web3().eth.contract(abi=router_abi)
    def decode_input_data(input_data):
        try:
            return generic_contract.decode_function_input(input_data[2:])[1] # Retorna apenas os params
        except: return {}
    df_raw['params'] = df_raw['inputData'].apply(decode_input_data)
    df_raw['path'] = df_raw['params'].apply(lambda p: p.get('path', []))
    df_raw['amountIn'] = df_raw['params'].apply(lambda p: p.get('amountIn', 0))
    # Vamos precisar do amountOut, mas para o algoritmo de busca de caminho, vamos usar uma aproximação
    df_raw['amountOut'] = df_raw['amountIn'] # ATENÇÃO: Isto é uma simplificação grosseira para encontrar os caminhos
    
    swaps_df = df_raw[df_raw['path'].apply(len) == 2].copy()
    swaps_df['token_in'] = swaps_df['path'].apply(lambda p: p[0])
    swaps_df['token_out'] = swaps_df['path'].apply(lambda p: p[1])
    # Calcula um preço implícito (simplificado)
    swaps_df['price'] = (swaps_df['amountOut'] + 1) / (swaps_df['amount_in'] + 1)

    print(f"Sucesso! Temos {len(swaps_df)} swaps diretos para a análise de ciclos.")

except Exception as e:
    print(f"--- ERRO: {e} ---")

# --- PASSO 2: Construir o Grafo e Caçar os Triângulos ---
if not swaps_df.empty:
    print("\n--- Passo 2: Construindo o grafo de preços e caçando ciclos ---")
    
    # Criamos um dicionário para representar nosso grafo de preços
    price_graph = {}
    for index, row in swaps_df.iterrows():
        if row['token_in'] not in price_graph:
            price_graph[row['token_in']] = {}
        price_graph[row['token_in']][row['token_out']] = row['price']

    # O Algoritmo Caçador de Triângulos
    found_opportunities = []
    
    # Para cada token A no nosso mapa...
    for token_a, connections in price_graph.items():
        # Para cada token B que A pode ser trocado...
        for token_b, price_ab in connections.items():
            # Verifica se B também tem conexões
            if token_b in price_graph:
                # Para cada token C que B pode ser trocado...
                for token_c, price_bc in price_graph[token_b].items():
                    # A condição de ouro: existe um caminho de volta de C para A?
                    if token_c in price_graph and token_a in price_graph[token_c]:
                        price_ca = price_graph[token_c][token_a]
                        
                        # Simula a arbitragem, incluindo uma taxa de 0.3% por swap
                        final_amount = 1 * price_ab * 0.997 * price_bc * 0.997 * price_ca * 0.997
                        
                        if final_amount > 1: # Se o resultado for maior que 1, encontramos lucro!
                            profit_percent = (final_amount - 1) * 100
                            opportunity = {
                                'path': f"{token_a[:10]}... -> {token_b[:10]}... -> {token_c[:10]}... -> {token_a[:10]}...",
                                'profit_perc': f"{profit_percent:.4f}%"
                            }
                            found_opportunities.append(opportunity)

    if found_opportunities:
        print(f"\nSucesso! Encontramos {len(found_opportunities)} oportunidades de arbitragem triangular em seus dados:")
        # Ordena pelas mais lucrativas
        top_opportunities = sorted(found_opportunities, key=lambda x: x['profit_perc'], reverse=True)
        for opp in top_opportunities[:10]: # Mostra as 10 melhores
            print(f"  - Caminho: {opp['path']}, Lucro Estimado: {opp['profit_perc']}")
    else:
        print("\nNenhuma oportunidade de arbitragem triangular encontrada no dataset.")

else:
    print("\nNenhum swap encontrado para analisar.")

--- Passo 1: Carregando e decodificando dados da Mainnet ---
--- ERRO: 'amount_in' ---

Nenhum swap encontrado para analisar.
